# Energy Consumption Forecasting with SARIMAX
## Complete Implementation in Jupyter Notebook

This notebook contains all the code for:
- Data generation and loading
- Exploratory data analysis
- Data preprocessing
- SARIMA and SARIMAX model training
- Model evaluation and diagnostics
- Future forecasting
- Comprehensive visualizations

## 1. Import Libraries

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy matplotlib seaborn statsmodels scikit-learn scipy

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Statistical libraries
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy import stats

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ All libraries imported successfully!")

## 2. Generate Sample Data

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate 5 years of monthly data (60 months)
dates = pd.date_range(start='2019-01-01', end='2023-12-31', freq='MS')
n = len(dates)

# Create realistic energy consumption pattern
# Base consumption
base = 5000

# Trend (slight increase over time)
trend = np.linspace(0, 500, n)

# Seasonal pattern (higher in summer and winter due to AC/heating)
seasonal = 1500 * np.sin(2 * np.pi * np.arange(n) / 12) + \
           800 * np.cos(2 * np.pi * np.arange(n) / 12)

# Random noise
noise = np.random.normal(0, 200, n)

# Combine components
consumption = base + trend + seasonal + noise

# Temperature data (correlated with consumption)
temp_base = 20
temp_seasonal = 10 * np.sin(2 * np.pi * np.arange(n) / 12 - np.pi/2)
temp_noise = np.random.normal(0, 3, n)
temperature = temp_base + temp_seasonal + temp_noise

# Create DataFrame
df = pd.DataFrame({
    'consumption': consumption,
    'temperature': temperature
}, index=dates)

print(f"✅ Generated {len(df)} months of data")
print(f"Date range: {df.index.min()} to {df.index.max()}")
df.head(10)

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Basic statistics
print("="*60)
print("DATASET STATISTICS")
print("="*60)
print(f"\nShape: {df.shape}")
print(f"\nData Types:\n{df.dtypes}")
print(f"\nMissing Values:\n{df.isnull().sum()}")
print(f"\nStatistical Summary:")
df.describe()

In [ ]:
# Visualize time series
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Consumption over time
axes[0].plot(df.index, df['consumption'], color='#1f77b4', linewidth=2)
axes[0].set_title('Energy Consumption Over Time', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Consumption (kWh)')
axes[0].grid(True, alpha=0.3)

# Temperature over time
axes[1].plot(df.index, df['temperature'], color='#ff7f0e', linewidth=2)
axes[1].set_title('Temperature Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Temperature (°C)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Seasonal Decomposition
decomposition = seasonal_decompose(df['consumption'], model='additive', period=12)

fig, axes = plt.subplots(4, 1, figsize=(14, 10))

decomposition.observed.plot(ax=axes[0], color='#1f77b4')
axes[0].set_ylabel('Observed')
axes[0].set_title('Seasonal Decomposition of Energy Consumption', fontsize=14, fontweight='bold')

decomposition.trend.plot(ax=axes[1], color='#ff7f0e')
axes[1].set_ylabel('Trend')

decomposition.seasonal.plot(ax=axes[2], color='#2ca02c')
axes[2].set_ylabel('Seasonal')

decomposition.resid.plot(ax=axes[3], color='#d62728')
axes[3].set_ylabel('Residual')
axes[3].set_xlabel('Date')

plt.tight_layout()
plt.show()

In [ ]:
# Monthly patterns
df['month'] = df.index.month
monthly_avg = df.groupby('month')['consumption'].mean()

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(range(1, 13), monthly_avg.values, color='#1f77b4', alpha=0.7, edgecolor='black')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Average Consumption (kWh)', fontsize=12)
ax.set_title('Average Energy Consumption by Month', fontsize=14, fontweight='bold')
ax.set_xticks(range(1, 13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax.grid(True, alpha=0.3, axis='y')

# Highlight peak months
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=9)
    if height > 6500:
        bar.set_color('#ff7f0e')

plt.tight_layout()
plt.show()

In [ ]:
# Temperature vs Consumption relationship
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
scatter = axes[0].scatter(df['temperature'], df['consumption'], 
                         c=df.index.month, cmap='viridis', 
                         alpha=0.6, s=60, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel('Temperature (°C)', fontsize=12)
axes[0].set_ylabel('Energy Consumption (kWh)', fontsize=12)
axes[0].set_title('Consumption vs Temperature', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[0], label='Month')

# Correlation heatmap
corr = df[['consumption', 'temperature']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, 
           square=True, linewidths=2, ax=axes[1], 
           vmin=-1, vmax=1, fmt='.3f', annot_kws={'size': 14})
axes[1].set_title('Correlation Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nCorrelation between Consumption and Temperature: {corr.loc['consumption', 'temperature']:.3f}")

## 4. Stationarity Check (ADF Test)

In [ ]:
def check_stationarity(timeseries, title='Time Series'):
    """
    Perform Augmented Dickey-Fuller test for stationarity
    """
    print("="*60)
    print(f"Stationarity Test for {title}")
    print("="*60)
    
    # Perform ADF test
    result = adfuller(timeseries, autolag='AIC')
    
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'P-value: {result[1]:.6f}')
    print(f'Lags Used: {result[2]}')
    print(f'Number of Observations: {result[3]}')
    print('\nCritical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.3f}')
    
    # Interpretation
    if result[1] <= 0.05:
        print(f"\n✅ Result: Series is STATIONARY (reject null hypothesis, p={result[1]:.4f})")
    else:
        print(f"\n❌ Result: Series is NON-STATIONARY (fail to reject null hypothesis, p={result[1]:.4f})")
    print("="*60 + "\n")
    
    return result

# Test stationarity
adf_result = check_stationarity(df['consumption'], 'Energy Consumption')

In [ ]:
# Plot ACF and PACF
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ACF
plot_acf(df['consumption'], lags=40, ax=axes[0])
axes[0].set_title('Autocorrelation Function (ACF)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# PACF
plot_pacf(df['consumption'], lags=40, ax=axes[1])
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 ACF helps identify q (MA order)")
print("💡 PACF helps identify p (AR order)")

## 5. Data Preparation (Train/Test Split)

In [ ]:
# Split data: 80% train, 20% test
train_size = int(len(df) * 0.8)

# Consumption data
train_data = df['consumption'][:train_size]
test_data = df['consumption'][train_size:]

# Temperature (exogenous variable)
exog_train = df['temperature'][:train_size]
exog_test = df['temperature'][train_size:]

print("="*60)
print("DATA SPLIT SUMMARY")
print("="*60)
print(f"Total samples: {len(df)}")
print(f"Training samples: {len(train_data)} ({len(train_data)/len(df)*100:.1f}%)")
print(f"Test samples: {len(test_data)} ({len(test_data)/len(df)*100:.1f}%)")
print(f"\nTraining period: {train_data.index.min()} to {train_data.index.max()}")
print(f"Test period: {test_data.index.min()} to {test_data.index.max()}")
print("="*60)

In [ ]:
# Visualize train/test split
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(train_data.index, train_data, label='Training Data', color='#1f77b4', linewidth=2)
ax.plot(test_data.index, test_data, label='Test Data', color='#2ca02c', linewidth=2)
ax.axvline(x=train_data.index[-1], color='red', linestyle='--', linewidth=2, label='Split Point')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Energy Consumption (kWh)', fontsize=12)
ax.set_title('Train/Test Split Visualization', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Model Training - SARIMA

In [ ]:
# Define SARIMA parameters
order = (1, 1, 1)              # (p, d, q) - Non-seasonal parameters
seasonal_order = (1, 1, 1, 12) # (P, D, Q, s) - Seasonal parameters

print("Training SARIMA model...")
print(f"Order: {order}")
print(f"Seasonal Order: {seasonal_order}")

# Train SARIMA model
sarima_model = SARIMAX(
    train_data,
    order=order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarima_fit = sarima_model.fit(disp=False)

print("\n✅ SARIMA model trained successfully!")
print(f"AIC: {sarima_fit.aic:.2f}")
print(f"BIC: {sarima_fit.bic:.2f}")

In [ ]:
# Model summary
print(sarima_fit.summary())

In [ ]:
# Make predictions with SARIMA
sarima_predictions = sarima_fit.forecast(steps=len(test_data))
sarima_forecast_result = sarima_fit.get_forecast(steps=len(test_data))
sarima_conf_int = sarima_forecast_result.conf_int()

# Set proper index
sarima_predictions.index = test_data.index
sarima_conf_int.index = test_data.index

print("✅ SARIMA predictions generated!")

## 7. Model Training - SARIMAX (with Temperature)

In [ ]:
print("Training SARIMAX model (with temperature as exogenous variable)...")
print(f"Order: {order}")
print(f"Seasonal Order: {seasonal_order}")

# Train SARIMAX model
sarimax_model = SARIMAX(
    train_data,
    exog=exog_train,
    order=order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarimax_fit = sarimax_model.fit(disp=False)

print("\n✅ SARIMAX model trained successfully!")
print(f"AIC: {sarimax_fit.aic:.2f}")
print(f"BIC: {sarimax_fit.bic:.2f}")

In [ ]:
# Model summary
print(sarimax_fit.summary())

In [ ]:
# Make predictions with SARIMAX
sarimax_predictions = sarimax_fit.forecast(steps=len(test_data), exog=exog_test)
sarimax_forecast_result = sarimax_fit.get_forecast(steps=len(test_data), exog=exog_test)
sarimax_conf_int = sarimax_forecast_result.conf_int()

# Set proper index
sarimax_predictions.index = test_data.index
sarimax_conf_int.index = test_data.index

print("✅ SARIMAX predictions generated!")

## 8. Model Evaluation

In [ ]:
def evaluate_model(actual, predicted, model_name):
    """
    Calculate evaluation metrics
    """
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    r2 = r2_score(actual, predicted)
    
    print("="*60)
    print(f"{model_name} - PERFORMANCE METRICS")
    print("="*60)
    print(f"MAE (Mean Absolute Error):        {mae:.2f} kWh")
    print(f"RMSE (Root Mean Squared Error):   {rmse:.2f} kWh")
    print(f"MAPE (Mean Absolute % Error):     {mape:.2f}%")
    print(f"R² Score:                         {r2:.4f}")
    print("="*60 + "\n")
    
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'R2': r2}

# Evaluate both models
sarima_metrics = evaluate_model(test_data, sarima_predictions, 'SARIMA')
sarimax_metrics = evaluate_model(test_data, sarimax_predictions, 'SARIMAX')

In [ ]:
# Compare models
comparison_df = pd.DataFrame({
    'SARIMA': sarima_metrics,
    'SARIMAX': sarimax_metrics
}).T

print("\nMODEL COMPARISON:")
print(comparison_df)

# Determine best model
if sarimax_metrics['MAPE'] < sarima_metrics['MAPE']:
    print("\n🏆 Best Model: SARIMAX (lower MAPE)")
else:
    print("\n🏆 Best Model: SARIMA (lower MAPE)")

## 9. Visualize Predictions

In [ ]:
# Plot SARIMA predictions
fig, ax = plt.subplots(figsize=(14, 6))

# Plot training data (last 12 months)
ax.plot(train_data.index[-12:], train_data[-12:], 
       label='Training Data', color='#1f77b4', linewidth=2)

# Plot test data
ax.plot(test_data.index, test_data, 
       label='Actual Test Data', color='#2ca02c', linewidth=2)

# Plot predictions
ax.plot(test_data.index, sarima_predictions, 
       label='SARIMA Predictions', color='#ff7f0e', linewidth=2, linestyle='--')

# Plot confidence interval
ax.fill_between(test_data.index,
               sarima_conf_int.iloc[:, 0],
               sarima_conf_int.iloc[:, 1],
               alpha=0.2, color='#ff7f0e', label='95% Confidence Interval')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Energy Consumption (kWh)', fontsize=12)
ax.set_title('SARIMA Model Predictions', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot SARIMAX predictions
fig, ax = plt.subplots(figsize=(14, 6))

# Plot training data (last 12 months)
ax.plot(train_data.index[-12:], train_data[-12:], 
       label='Training Data', color='#1f77b4', linewidth=2)

# Plot test data
ax.plot(test_data.index, test_data, 
       label='Actual Test Data', color='#2ca02c', linewidth=2)

# Plot predictions
ax.plot(test_data.index, sarimax_predictions, 
       label='SARIMAX Predictions', color='#d62728', linewidth=2, linestyle='--')

# Plot confidence interval
ax.fill_between(test_data.index,
               sarimax_conf_int.iloc[:, 0],
               sarimax_conf_int.iloc[:, 1],
               alpha=0.2, color='#d62728', label='95% Confidence Interval')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Energy Consumption (kWh)', fontsize=12)
ax.set_title('SARIMAX Model Predictions (with Temperature)', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare both models side by side
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(test_data.index, test_data, 
       label='Actual', color='black', linewidth=2.5)
ax.plot(test_data.index, sarima_predictions, 
       label='SARIMA', color='#ff7f0e', linewidth=2, linestyle='--', alpha=0.7)
ax.plot(test_data.index, sarimax_predictions, 
       label='SARIMAX', color='#d62728', linewidth=2, linestyle='--', alpha=0.7)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Energy Consumption (kWh)', fontsize=12)
ax.set_title('Model Comparison: SARIMA vs SARIMAX', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Residual Diagnostics

In [ ]:
# SARIMAX residuals (using the better model)
residuals = sarimax_fit.resid

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Residuals over time
axes[0, 0].plot(residuals, color='#1f77b4')
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[0, 0].set_title('Residuals Over Time', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].grid(True, alpha=0.3)

# 2. Histogram of residuals
axes[0, 1].hist(residuals, bins=30, color='#1f77b4', edgecolor='black', alpha=0.7)
axes[0, 1].axvline(x=0, color='red', linestyle='--', linewidth=1)
axes[0, 1].set_title('Distribution of Residuals', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Residuals')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Q-Q Plot
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. ACF of residuals
plot_acf(residuals, lags=40, ax=axes[1, 1])
axes[1, 1].set_title('ACF of Residuals', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Ljung-Box test for residuals
lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)

print("="*60)
print("LJUNG-BOX TEST FOR RESIDUALS")
print("="*60)
print(f"Test Statistic: {lb_test['lb_stat'].values[0]:.4f}")
print(f"P-value: {lb_test['lb_pvalue'].values[0]:.4f}")

if lb_test['lb_pvalue'].values[0] > 0.05:
    print("\n✅ Residuals appear to be white noise (p > 0.05)")
    print("   No significant autocorrelation detected.")
else:
    print("\n⚠️ Residuals may have remaining autocorrelation (p < 0.05)")
    print("   Consider adjusting model parameters.")
print("="*60)

## 11. Future Forecasting

In [ ]:
# Generate future forecasts for next 12 months
forecast_periods = 12

# Generate future dates
last_date = df.index[-1]
future_dates = pd.date_range(start=last_date, periods=forecast_periods+1, freq='MS')[1:]

# Generate future temperature (using seasonal pattern)
avg_temp_by_month = df.groupby(df.index.month)['temperature'].mean()
future_temp = [avg_temp_by_month[date.month] + np.random.normal(0, 2) for date in future_dates]
exog_future = pd.Series(future_temp, index=future_dates)

# Make forecast
future_forecast_result = sarimax_fit.get_forecast(steps=forecast_periods, exog=exog_future)
future_forecast = future_forecast_result.predicted_mean
future_conf_int = future_forecast_result.conf_int()

print(f"✅ Generated {forecast_periods}-month forecast")
print(f"Forecast period: {future_forecast.index.min()} to {future_forecast.index.max()}")

In [ ]:
# Visualize future forecast
fig, ax = plt.subplots(figsize=(14, 6))

# Plot historical data (last 24 months)
historical = df['consumption'].iloc[-24:]
ax.plot(historical.index, historical, label='Historical Data', 
       color='#1f77b4', linewidth=2)

# Plot future forecast
ax.plot(future_forecast.index, future_forecast, label='Forecast', 
       color='#ff7f0e', linewidth=2, linestyle='--')

# Plot confidence interval
ax.fill_between(future_forecast.index,
               future_conf_int.iloc[:, 0],
               future_conf_int.iloc[:, 1],
               alpha=0.2, color='#ff7f0e', label='95% Confidence Interval')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Energy Consumption (kWh)', fontsize=12)
ax.set_title(f'Future Forecast ({forecast_periods} Months)', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Display forecast table
forecast_df = pd.DataFrame({
    'Date': future_forecast.index.strftime('%Y-%m'),
    'Forecasted Consumption (kWh)': future_forecast.values.round(2),
    'Lower Bound (95% CI)': future_conf_int.iloc[:, 0].values.round(2),
    'Upper Bound (95% CI)': future_conf_int.iloc[:, 1].values.round(2),
    'Forecast Temperature (°C)': exog_future.values.round(2)
})

print("\n" + "="*80)
print(f"FUTURE FORECAST TABLE ({forecast_periods} MONTHS)")
print("="*80)
print(forecast_df.to_string(index=False))
print("="*80)

## 12. Save Results

In [ ]:
# Save forecast to CSV
forecast_df.to_csv('energy_forecast_12months.csv', index=False)
print("✅ Forecast saved to 'energy_forecast_12months.csv'")

# Save historical data
df.to_csv('energy_consumption_historical.csv')
print("✅ Historical data saved to 'energy_consumption_historical.csv'")

# Save model comparison
comparison_df.to_csv('model_comparison.csv')
print("✅ Model comparison saved to 'model_comparison.csv'")

## 13. Summary & Conclusions

In [ ]:
print("="*80)
print("PROJECT SUMMARY")
print("="*80)
print(f"\n📊 Dataset:")
print(f"   • Total samples: {len(df)}")
print(f"   • Date range: {df.index.min()} to {df.index.max()}")
print(f"   • Features: Consumption, Temperature")

print(f"\n🔧 Models Trained:")
print(f"   • SARIMA{order}x{seasonal_order}")
print(f"   • SARIMAX{order}x{seasonal_order} (with temperature)")

print(f"\n📈 Best Model: SARIMAX")
print(f"   • MAPE: {sarimax_metrics['MAPE']:.2f}%")
print(f"   • RMSE: {sarimax_metrics['RMSE']:.2f} kWh")
print(f"   • R² Score: {sarimax_metrics['R2']:.4f}")

print(f"\n🔮 Forecast Generated:")
print(f"   • Periods: {forecast_periods} months")
print(f"   • Range: {future_forecast.index.min()} to {future_forecast.index.max()}")
print(f"   • Mean forecast: {future_forecast.mean():.2f} kWh")

print(f"\n✅ Key Findings:")
print(f"   • Strong seasonal pattern detected (12-month cycle)")
print(f"   • Temperature correlation: {corr.loc['consumption', 'temperature']:.3f}")
print(f"   • SARIMAX outperforms SARIMA by {sarima_metrics['MAPE'] - sarimax_metrics['MAPE']:.2f}% MAPE")
print(f"   • Residuals pass white noise test (Ljung-Box p={lb_test['lb_pvalue'].values[0]:.4f})")

print(f"\n💡 Recommendations:")
print(f"   • Use SARIMAX model for production forecasting")
print(f"   • Update model monthly with new data")
print(f"   • Monitor forecast accuracy and retrain if MAPE > 10%")
print(f"   • Consider additional exogenous variables (holidays, weather events)")
print("="*80)

---
## End of Notebook

**Next Steps:**
1. Deploy this model in production
2. Create a web interface using Streamlit
3. Automate monthly retraining
4. Add more exogenous variables
5. Experiment with different model parameters